# README
- This is an example for DNMT3A which is based on a CBE screen
- All settings are set with default values as below


# Input

In [2]:
# @markdown # Required (filled with default values for DNMT3A)
# @markdown Input Gene and UniprotID (for AF2 structure)
input_gene = 'DNMT3A' # @param {type:"string"}
input_uniprot = 'Q9Y6K1' # @param {type:"string"}
structureid = f"AF-{input_uniprot}-F1-model_v4"

# @markdown Screen File (TSV)
screen_dir = '' # @param {type:"string"}
screens = 'LueNatChemBiol2022-DNMT3A-CBE.tsv' # @param {type:"string"}

mut_list_col = ""
mut_col = "Mut_type" # @param {type:"string"}
val_col = "sgRNA_score" # @param {type:"string"}
gene_col = "Gene" # @param {type:"string"}
edits_col = "Mutation_list" # @param {type:"string"}
gRNA_col = "gRNA_Seq" # @param {type:"string"}

# @markdown ---

# @markdown # Optional
# @markdown
function_for_lfc = 'max' #@param
function_for_lfc3d = 'mean' #@param
nRandom = 500 #@param
pthr = 0.05 #@param
single_screen_pthr = pthr
single_screen_pthr_str = str(single_screen_pthr).split('.')[1]

# @markdown Where files and plots are stored:
workdir = '' # @param {type:"string"}
job_id = '' # @param {type:"string"}

# @markdown Radius to calculate LFC3D over:
structure_radius = 6.0 # @param {type:"number"}
# @markdown Radius to cluster over:
clustering_radius = 6.0 # @param {type:"number"}

# @markdown Optional user-provided filepaths
# user_uniprot = 'Q9Y6K1' # @param {type:"string"}
user_pdb = None # @param {type:"string"}
user_dssp = None # @param {type:"string"}
chain = 'A' # @param {type:"string"}

## Setup

In [3]:
# @title Connect to Google Drive
! apt-get update
! apt-get install dssp clustalo
! wget https://raw.githubusercontent.com/broadinstitute/BE3D/refs/heads/main/data/LueNatChemBiol2022-DNMT3A-CBE.tsv -O LueNatChemBiol2022-DNMT3A-CBE.tsv

from google.colab import drive
drive.mount('/content/drive')


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,267 kB]
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,840 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubun

In [4]:
# @title Install BEClust3D

! pip install git+https://github.com/broadinstitute/beclust3d-public.git


  Cloning https://github.com/broadinstitute/beclust3d-public.git to /tmp/pip-req-build-b72w6djd
  Running command git clone --filter=blob:none --quiet https://github.com/broadinstitute/beclust3d-public.git /tmp/pip-req-build-b72w6djd
  Resolved https://github.com/broadinstitute/beclust3d-public.git to commit f4f4f1f393e78efe32acb5860db40a89b1cf4499
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.3/68.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 44.5 MB/s eta 0:00:00
  Created wheel for beclust3d: filename=beclust3d-2025.0.0-py3-none-any.whl size=1315856 sha256=5e3315c31298515635cb016f1b8f82555f81f70aefc5d8fe008506c9853ba9fa
  Stored in directory: /tmp/pip-ephem-wheel-cache-00cy242g/wheels/8e/41/d6/2ee7d7b230909d91878a638fe7d9d41242d670dd15345d106c
  Created wheel for wget: filen

## Import

In [5]:
# @title Import packages and set variables
import os
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import ipywidgets as widgets
from IPython.display import display
import gc
import uuid

from beclust3d import *
from IPython.display import Image, display, SVG
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

if len(screens) == 0:
    screens = [f for f in os.listdir(screen_dir) if os.path.isfile(workdir+screen_dir + '/' + f)]
else:
    screens = screens.split(', ')

screen_names = [s.split('.')[0] for s in screens]
input_dfs = [pd.read_csv(screen_dir+s, sep='\t') for s in screens]

if len(job_id) == 0:
    job_id = str(uuid.uuid4())
    print(f"Job ID: {job_id}")

output_dir = os.path.join(workdir, f'{input_gene}-Job-{job_id}')
os.makedirs(output_dir, exist_ok=True)
print('All results will be saved in the following directory:')
print(output_dir)

Job ID: 79f21cd6-1170-4b85-ab12-fffa8c0d1fc4
All results will be saved in the following directory:
DNMT3A-Job-79f21cd6-1170-4b85-ab12-fffa8c0d1fc4


# Running BE3D

## Running parts

In [ ]:
#@title running
struc_df = sequence_structural_features(
    output_dir,
    input_gene, input_uniprot, structureid,
    user_pdb=user_pdb,
    user_dssp=user_dssp,
    chains=[chain],
    radius=structure_radius,
    )


df_residuemap = pd.DataFrame()
conserv_dfs = list()
gene_list = list()


for screen_name in screen_names:
      conserv_dfs.append(None)
      gene_list.append(input_gene)

# Only for Human Gene
sequence_structural_features(
    output_dir,
    input_gene, input_uniprot, structureid,
    user_fasta=None,
    user_pdb=user_pdb,
    user_dssp=user_dssp,
    chains=['A'],
    radius=structure_radius,
    )

# For All gene
hypothesis_test(
    output_dir,
    input_dfs, screen_names,
    cases=['Nonsense', 'Splice Site'],
    controls=['Silent', 'No Mutation'],
    comp_name='Nonsense_Splice_vs_SilentNoMut',
    mut_col=mut_col,
    val_col=val_col,
    gene_col=gene_col,
    save_type='svg'
    )


h2_ks_test_pd = pd.read_csv(f'{output_dir}/hypothesis_qc/KolmogorovSmirnov_hypothesis2.tsv',sep='\t')
h2_ks_test_pd = h2_ks_test_pd.replace(-999,None)
white_screen_list = h2_ks_test_pd[(h2_ks_test_pd['p_Nonsense_Splice_vs_SilentNoMut']<0.05)&(h2_ks_test_pd['gene_name'].isin(gene_list))]['screenid'].to_list()
print(f'original screen size: {len(screen_names)}, screen white list size: {len(white_screen_list)}, QA-passed screen size: {len(list(set(screen_names).intersection(white_screen_list)))}')
# print(gene_list)
screen_names = list(set(screen_names).intersection(white_screen_list))
input_dfs = [pd.read_csv(os.path.join(screen_dir,f'{s}.tsv'), sep='\t') for s in screen_names]


# For All
parse_be_data(
    output_dir,
    input_dfs, input_gene, screen_names,
    mut_col=mut_col,
    val_col=val_col,
    gene_col=gene_col,
    edits_col=edits_col,
    mut_categories = ["Nonsense", "Splice Site", "Missense", "No Mutation", "Silent"],
    mut_delimiter = ';',
    # conserv_dfs = [None for _ in screen_names]
    conserv_dfs = conserv_dfs,
    conserv_col='mouse_res_pos',
    gene_list=gene_list
    )

# For All
plot_rawdata(
    output_dir,
    input_dfs,
    screen_names,
    mut_col=mut_col,
    val_col=val_col,
    gene_col=gene_col,
    mut_categories = ["Nonsense", "Splice Site", "Missense", "No Mutation", "Silent"],
    )

# For All
df_missense_list = [
    pd.read_csv(f'{output_dir}/screendata/{gene}_{screen_name}_Missense.tsv',
                sep='\t') for gene, screen_name in zip(gene_list, screen_names)
]

# For All
for df_missense, screen_name, gene in zip(df_missense_list, screen_names, gene_list):
    randomize_data(
        df_missense,
        output_dir, gene,
        screen_name,
        nRandom=nRandom,
        seed=True,
        )

df_struc = pd.read_csv(f'{output_dir}/sequence_structure/{structureid}_coord_struc_features.tsv', sep='\t')
# Human and Non-human mixed
for gene, screen_name, df_consrv in zip(gene_list, screen_names, conserv_dfs):
    df_control = pd.read_csv(f'{output_dir}/screendata/{gene}_{screen_name}_No_Mutation.tsv', sep='\t', index_col=0)
    df_dict = {}

    for mut in ['Missense', 'Silent', 'Nonsense']:
        filepath = f'{output_dir}/screendata/{gene}_{screen_name}_{mut}.tsv'
        if os.path.exists(filepath):
            df_dict[mut] = pd.read_csv(filepath, sep='\t', index_col=0)

    if df_consrv is not None:
        target_res_pos, target_res = 'human_res_pos', 'unires'
        alternate_res_pos, alternate_res = 'mouse_res_pos','mouse_res'
        df_missense = prioritize_by_sequence(
            df_dict,
            df_struc, df_consrv, df_control,
            output_dir,
            gene, screen_name,
            target_res_pos=target_res_pos,
            alt_res_pos=alternate_res_pos,
            alt_res=alternate_res
        )
    else:
        df_missense = prioritize_by_sequence(
            df_dict,
            df_struc, df_consrv, df_control,
            output_dir,
            gene, screen_name
        )

    df_rand = pd.read_csv(f'{output_dir}/screendata_rand/{gene}_{screen_name}_Missense_rand.tsv', sep='\t')

    # For ALL
    randomize_sequence(
        df_missense, df_rand,
        output_dir,
        gene, screen_name,
        nRandom=nRandom, conservation=False,
        muttype='Missense',
        function_name=function_for_lfc,
        target_pos='unipos', target_res=None,
        )

    plot_screendata_sequence(
        df_missense,
        output_dir,
        gene, screen_name, function_name=function_for_lfc, muttype='Missense',
    )

def run_Clust3D_per_species(gene, screen_names):
  ## CLUSTERING ON PRIORITIZED
  df_struc = pd.read_csv(f'{output_dir}/sequence_structure/{structureid}_coord_struc_features.tsv', sep='\t')
  ## LFC3D
  df_edits_list = []
  df_rand_list = []

  for screen_name in screen_names:
      df_missense = pd.read_csv(f'{output_dir}/screendata_sequence/{gene}_{screen_name}_protein_edits.tsv', sep='\t')
      df_missense[f'{function_for_lfc}_Missense_LFC_plab_input'] = df_missense[f'{function_for_lfc}_Missense_LFC_p'].apply(lambda x: f'p<{single_screen_pthr_str}' if x < single_screen_pthr else f'p>={single_screen_pthr_str}')

      df_hits_clust, distances, yvalues = clustering(
          df_struc, df_missense,
          output_dir, gene,
          psig_columns=[f'{function_for_lfc}_Missense_LFC_plab_input'],
          pthr_cutoffs=[f'p<{single_screen_pthr_str}'], # TODO:
          screen_name=screen_name, score_type='lfc',
          max_distances=25, merge_cols=['unipos', 'chain'],
      )

      df_protein_edits = pd.read_csv(f'{output_dir}/screendata_sequence/{gene}_{screen_name}_protein_edits.tsv', sep='\t')
      df_edits_list.append(df_protein_edits)
      df_protein_edits_rand = pd.read_csv(f'{output_dir}/screendata_sequence_rand/{gene}_{screen_name}_Missense_protein_edits_rand.tsv', sep='\t')
      df_rand_list.append(df_protein_edits_rand)

  df_LFC_LFC3D = calculate_lfc3d(
      df_struc, df_edits_list, df_rand_list,
      output_dir, gene, screen_names,
      nRandom=nRandom,  muttype='Missense',
      function_type_lfc=function_for_lfc,
      function_type_lfc3d=function_for_lfc3d,
      conserved_only=False,
  )

  # LFC #
  df_bidir = average_split_score(
      df_LFC_LFC3D,
      output_dir, gene, screen_names,
      score_type='LFC',
  )
  df_dis, _, _ = bin_score(
      df_bidir,
      output_dir, gene, screen_names,
      score_type='LFC',
  )

  znorm_score(
      df_bidir, output_dir, gene, screen_names,
      pthrs=[0.05, 0.01, 0.001], score_type='LFC',
  )

  df_lfc = pd.read_csv(f'{output_dir}/LFC/{gene}_NonAggr_LFC.tsv', sep='\t')

  for screen_name in screen_names:
      average_split_bin_plots(
          df_lfc,
          workdir = output_dir,
          input_gene = gene,
          screen_name=screen_name, # BLANK FOR META #
          func='', # BLANK FOR NON AGGR #
          pthr=single_screen_pthr,
          score_type='LFC',
          aggregate_dir='LFC',
          save_type='svg'
          )

  df_bidir = average_split_score(
      df_LFC_LFC3D,
      output_dir, gene, screen_names,
      score_type='LFC3D',
  )
  df_dis, _, _ = bin_score(
      df_bidir,
      output_dir, gene, screen_names,
      score_type='LFC3D',
  )

  znorm_score(
      df_bidir, output_dir, gene, screen_names,
      pthrs=[0.05, 0.01, 0.001], score_type='LFC3D'
  )

  df_lfc3d = pd.read_csv(f'{output_dir}/LFC3D/{gene}_NonAggr_LFC3D.tsv', sep='\t')

  for screen_name in screen_names:
      average_split_bin_plots(
          df_lfc3d,
          workdir = output_dir,
          input_gene = gene,
          screen_name=screen_name, # BLANK FOR META #
          func='', # BLANK FOR NON AGGR #
          pthr=single_screen_pthr,
          score_type='LFC3D',
          aggregate_dir='LFC3D',
          save_type='svg'
          )

  for score_type in ['LFC', 'LFC3D']:
      df_pvals = pd.read_csv(f'{output_dir}/{score_type}/{gene}_NonAggr_{score_type}.tsv', sep='\t')

      for screen_name in screen_names:
          df_hits_clust, distances, yvalues = clustering(
              df_struc, df_pvals,
              output_dir, gene,
              # psig_columns=[f'{screen_name}_{score_type}_neg_{pthr_str_short}_psig',
              #             f'{screen_name}_{score_type}_pos_{pthr_str_short}_psig'],
              # pthr_cutoffs=[f'p<{pthr_str}', f'p<{pthr_str}'],
              psig_columns=[f'{screen_name}_{score_type}_neg_05_psig',
                          f'{screen_name}_{score_type}_pos_05_psig',
                          f'{screen_name}_{score_type}_neg_01_psig',
                          f'{screen_name}_{score_type}_pos_01_psig',
                          f'{screen_name}_{score_type}_neg_001_psig',
                          f'{screen_name}_{score_type}_pos_001_psig'
                          ],
              pthr_cutoffs=['p<0.05', 'p<0.05',
                          'p<0.01', 'p<0.01',
                          'p<0.001', 'p<0.001',
                          ],
              screen_name=screen_name, score_type=score_type,
              max_distances=20, merge_cols=['unipos', 'chain'],
          )

          # PLOTTING #
          plot_clustering(
              df_struc, df_pvals,
              df_hits_clust, clustering_radius,
              output_dir, gene,
              distances, yvalues,
              # psig_columns=[f'{screen_name}_{score_type}_neg_{pthr_str_short}_psig',
              #             f'{screen_name}_{score_type}_pos_{pthr_str_short}_psig'],
              # names=['Negative', 'Positive'],
              names=['Negative', 'Positive', 'Negative', 'Positive', 'Negative', 'Positive'],
              # pthr_cutoffs=[f'p<{pthr_str}', f'p<{pthr_str}'],
              psig_columns=[f'{screen_name}_{score_type}_neg_05_psig',
                          f'{screen_name}_{score_type}_pos_05_psig',
                          f'{screen_name}_{score_type}_neg_01_psig',
                          f'{screen_name}_{score_type}_pos_01_psig',
                          f'{screen_name}_{score_type}_neg_001_psig',
                          f'{screen_name}_{score_type}_pos_001_psig'
                          ],
              pthr_cutoffs=['p<0.05', 'p<0.05',
                          'p<0.01', 'p<0.01',
                          'p<0.001', 'p<0.001',
                          ],
              screen_name=screen_name, score_type=score_type,
              merge_col=['unipos', 'chain'],
              save_type='svg',
              dendrogram_subplots_kwargs={'figsize':(15, 3.5)}
          )

  for screen_name in screen_names:
      # LFC vs LFC3D SCATTERPLOT #
      cutoff_str = str(single_screen_pthr).split('.')[1]
      df_lfc = pd.read_csv(f"{output_dir}/LFC/{gene}_NonAggr_LFC.tsv", sep='\t')
      df_lfc_dis = pd.read_csv(f"{output_dir}/LFC/{gene}_LFC_dis_wght.tsv", sep='\t')
      df_lfc3d = pd.read_csv(f"{output_dir}/LFC3D/{gene}_NonAggr_LFC3D.tsv", sep='\t')
      df_lfc3d_dis = pd.read_csv(f"{output_dir}/LFC3D/{gene}_LFC3D_dis_wght.tsv", sep='\t')

      df_dis_input = pd.DataFrame()
      df_dis_input['unipos'] = df_lfc_dis['unipos']

      df_dis_input = pd.concat([df_dis_input,df_lfc_dis.filter(regex=r'LFC$'),df_lfc3d_dis.filter(regex=r'LFC3D$'),df_lfc3d.filter(regex=fr'LFC3D_dis$|{cutoff_str}_psig$')])
      df_dis_input = df_dis_input.rename({f'{screen_name}_LFC3D_neg_{cutoff_str}_psig':f'{screen_name}_LFC3D_neg_psig',
                                          f'{screen_name}_LFC3D_pos_{cutoff_str}_psig':f'{screen_name}_LFC3D_pos_psig',
                                          },axis=1)
      lfc_lfc3d_scatter(
          df_input=df_dis_input,
          workdir=output_dir,
          input_gene=gene, screen_name=screen_name,
          pthr=single_screen_pthr,
      )
      os.rename(f'{output_dir}/characterization/plots/{gene}_LFC_LFC3D_scatter.png',
              f'{output_dir}/characterization/plots/{gene}_LFC_LFC3D_scatter_{cutoff_str}_{screen_name}.png')

  # Load both LFC and lFC3D dataframes
  df_pvals_LFC3D = pd.read_csv(f'{output_dir}/LFC3D/{gene}_NonAggr_LFC3D.tsv', sep='\t')
  df_pvals_LFC = pd.read_csv(f'{output_dir}/LFC/{gene}_NonAggr_LFC.tsv', sep='\t')
  df_pvals = pd.concat([df_pvals_LFC3D, df_pvals_LFC.drop(columns=['unipos', 'unires', 'chain'])], axis=1)

  def find_union(input, pthr_str):
    if input[0] == f'p<{pthr_str}' or input[1] == f'p<{pthr_str}':
        return f'p<{pthr_str}'
    else:
        return f'p>={pthr_str}'

  # Find union of LFC and LFC3D
  for screen_name in screen_names:
      for each_pthr in ['05','01','001']:
          df_pvals[f'{screen_name}_union_neg_{each_pthr}_psig'] = df_pvals[[f'{screen_name}_LFC_neg_{each_pthr}_psig', f'{screen_name}_LFC3D_neg_{each_pthr}_psig']].apply(lambda row: find_union(row, f'0.{each_pthr}'), axis=1)
          df_pvals[f'{screen_name}_union_pos_{each_pthr}_psig'] = df_pvals[[f'{screen_name}_LFC_pos_{each_pthr}_psig', f'{screen_name}_LFC3D_pos_{each_pthr}_psig']].apply(lambda row: find_union(row, f'0.{each_pthr}'), axis=1)

      df_hits_clust, distances, yvalues = clustering(
          df_struc, df_pvals,
          output_dir, gene,
          psig_columns=[f'{screen_name}_union_neg_05_psig',
                      f'{screen_name}_union_pos_05_psig',
                      f'{screen_name}_union_neg_01_psig',
                      f'{screen_name}_union_pos_01_psig',
                      f'{screen_name}_union_neg_001_psig',
                      f'{screen_name}_union_pos_001_psig'
                      ],
          pthr_cutoffs=['p<0.05', 'p<0.05',
                      'p<0.01', 'p<0.01',
                      'p<0.001', 'p<0.001',
                      ],
          screen_name=screen_name, score_type='union',
          max_distances=20, merge_cols=['unipos', 'chain'],
      )

      # PLOTTING #
      plot_clustering(
          df_struc, df_pvals,
          df_hits_clust, clustering_radius,
          output_dir, gene,
          distances, yvalues,
          names=['Negative', 'Positive', 'Negative', 'Positive', 'Negative', 'Positive'],
          psig_columns=[f'{screen_name}_union_neg_05_psig',
                      f'{screen_name}_union_pos_05_psig',
                      f'{screen_name}_union_neg_01_psig',
                      f'{screen_name}_union_pos_01_psig',
                      f'{screen_name}_union_neg_001_psig',
                      f'{screen_name}_union_pos_001_psig'
                      ],
          pthr_cutoffs=['p<0.05', 'p<0.05',
                      'p<0.01', 'p<0.01',
                      'p<0.001', 'p<0.001',
                      ],
          screen_name = screen_name, score_type='union',
          merge_col=['unipos', 'chain'],
          save_type='svg',
          dendrogram_subplots_kwargs={'figsize':(15, 3.5)}
      )

  return df_LFC_LFC3D

df_LFC_LFC3D_original, df_LFC_LFC3D_alt, merged_df_LFC_LFC3D = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
gene = input_gene
merged_df_LFC_LFC3D =  run_Clust3D_per_species(input_gene, [x[1] for x in zip(conserv_dfs, screen_names) if x[0] is None]) # For human
g2p_formatted_hit_cluster(output_dir, input_gene, screen_names, lfc_pthr=single_screen_pthr_str, lfc3d_pthr=single_screen_pthr_str, meta_pthr='001', meta=False)


sequence_structure/Q9Y6K1.fasta exists
original screen size: 1, screen white list size: 1, QA-passed screen size: 1
Processing LueNatChemBiol2022-DNMT3A-CBE
{'Nonsense': 18, 'Splice Site': 12, 'Missense': 204, 'No Mutation': 166, 'Silent': 88}
Calculated LFC3D for LueNatChemBiol2022-DNMT3A-CBE


# Analysis

In [ ]:
# @title LFC, LFC3D Residue-dot Plot
# Load LFC data
lfc_pd = pd.read_csv(f'{output_dir}/LFC/DNMT3A_NonAggr_LFC.tsv', sep='\t')
lfc_pd = lfc_pd.replace('-', None)
lfc_pd['LueNatChemBiol2022-DNMT3A-CBE_LFC_pos'] = pd.to_numeric(lfc_pd['LueNatChemBiol2022-DNMT3A-CBE_LFC_pos'], errors='coerce')

# Load LFC3D data
lfc3d_pd = pd.read_csv(f'{output_dir}/LFC3D/DNMT3A_NonAggr_LFC3D.tsv', sep='\t')
lfc3d_pd = lfc3d_pd.replace('-', None)
lfc3d_pd['LueNatChemBiol2022-DNMT3A-CBE_LFC3D_pos'] = pd.to_numeric(lfc3d_pd['LueNatChemBiol2022-DNMT3A-CBE_LFC3D_pos'], errors='coerce')

# Map significance to color arrays
lfc_colors = lfc_pd['LueNatChemBiol2022-DNMT3A-CBE_LFC_pos_05_psig'].map({'p<0.05': 'blue'}).fillna('gray')
lfc3d_colors = lfc3d_pd['LueNatChemBiol2022-DNMT3A-CBE_LFC3D_pos_05_psig'].map({'p<0.05': 'blue'}).fillna('gray')

# Shared axis ranges
x_min = min(lfc_pd['unipos'].min(), lfc3d_pd['unipos'].min())
x_max = max(lfc_pd['unipos'].max(), lfc3d_pd['unipos'].max())
y_min = min(lfc_pd['LueNatChemBiol2022-DNMT3A-CBE_LFC_pos'].min(), lfc3d_pd['LueNatChemBiol2022-DNMT3A-CBE_LFC3D_pos'].min())
y_max = max(lfc_pd['LueNatChemBiol2022-DNMT3A-CBE_LFC_pos'].max(), lfc3d_pd['LueNatChemBiol2022-DNMT3A-CBE_LFC3D_pos'].max())

# Create subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=('LFC', 'LFC3D'),
                    shared_xaxes=True, shared_yaxes=True)

# Plot LFC
fig.add_trace(go.Scatter(
    x=lfc_pd['unipos'],
    y=lfc_pd['LueNatChemBiol2022-DNMT3A-CBE_LFC_pos'],
    mode='markers',
    marker=dict(color=lfc_colors, size=10),
    showlegend=False
), row=1, col=1)

# Plot LFC3D
fig.add_trace(go.Scatter(
    x=lfc3d_pd['unipos'],
    y=lfc3d_pd['LueNatChemBiol2022-DNMT3A-CBE_LFC3D_pos'],
    mode='markers',
    marker=dict(color=lfc3d_colors, size=10),
    showlegend=False
), row=1, col=2)

# # Add horizontal reference lines at y=0
# for col in [1, 2]:
#     fig.add_shape(type='line',
#                   x0=x_min, x1=x_max,
#                   y0=0, y1=0,
#                   line=dict(color='black', width=1, dash='dot'),
#                   row=1, col=col)

# Add manual legend (dummy invisible points with legend)
fig.add_trace(go.Scatter(
    x=[None], y=[None],
    mode='markers',
    marker=dict(size=10, color='blue'),
    name='p < 0.05'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=[None], y=[None],
    mode='markers',
    marker=dict(size=10, color='gray'),
    name='p > 0.05'
), row=1, col=1)

# Layout and axes
fig.update_layout(
    height=600,
    width=1400,
    title_text="DNMT3A Residue-level LFC and LFC3D Plot",
    legend=dict(x=1.05, y=1, traceorder="normal")
)

fig.update_xaxes(title_text="Residue Position", range=[x_min, x_max])
fig.update_yaxes(title_text="Signal", range=[y_min, y_max+0.3])

fig.show()

In [ ]:
# @title QA Bar Plot
# display(SVG(f'/content/{output_dir}/hypothesis_qc/hypothesis2_scatterplot_by_screenid.svg'))
display(Image(filename=f'/content/{output_dir}/screendata/plots/LueNatChemBiol2022-DNMT3A-CBE_barplot_by_muttype.png'))


In [ ]:
# @title QA Violin Plot
display(Image(filename=f'{output_dir}/screendata/plots/LueNatChemBiol2022-DNMT3A-CBE_violinplot_by_muttype.png'))

In [ ]:
# @title LFC3D Hit Clusters (Dendrogram)
display(SVG(filename=f'{output_dir}/cluster_LFC3D/plots/DNMT3A_LueNatChemBiol2022-DNMT3A-CBE_LFC3D_Positive_Dendrogram_6A.svg'))